In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
import yfinance as yf
import pickle
import requests
from requests.auth import HTTPBasicAuth
import hashlib
from urllib.parse import urlencode
import hmac
import math

%run helper_functions.ipynb
%run train.ipynb

Using TensorFlow backend.


Testing set size: 430
Model guessed up or down movement correctly 0.6116279069767442%
mae: 0.02138560087627802
mse: 0.001098144583812084
R^2: 0.28997025536284504


# Get data
Get latest data and features to predict next days price

In [2]:
hist = get_yfinance_data('BTC-USD')[['Close']].rename(columns = {'Close': 'Btc Close'}) # get btc hist
hist['Btc Returns'] = hist['Btc Close'].pct_change() # get btc returns

# Lagged Moving average - Btc price
windows = [3, 5, 7, 10, 20, 50, 100, 200] # set moving average windows
for window in windows:
    hist['Btc Close - ' +str(window) + 
         'd Avg'] = hist['Btc Close'] - moving_average(hist['Btc Close'], window=window, lag=0)

# Predict and Trade
Connect to API <br>
Make trade based on prediction. <br>
Scenario 1: Positive return pred. If in bitcoin, stay. Else out of bitcoin, buy. <br>
Scenario 2: Negative return pred. If in bitcoin, sell. Else out of bitcoin, stay. <br>

### Load Model and predict

In [3]:
loaded_model = pickle.load(open('model_weights.sav', 'rb')) # load model
pred = loaded_model.predict(hist[hist.columns[2:]].tail(1)) # make prediction on latest data

### Set API variables

In [4]:
api_key = {'X-MBX-APIKEY' : 'Bc4rxmrzUb5r9Q3Mmjw18SdtG6n67Dgj757AKgmQVUOmtY5K0pVHr6q8XSFyNPA2'} # Set API key
sec_key = 'bjZYr2spX2LzeJNfzgoaSpLtYnixhM9ko1ik9N6WZ3Ey2JzC0li8OlRwegImEUQ0' # Set API secret
account_endpoint = 'https://api.binance.com/api/v3/account'  # account balances endpoint
price_endpoint = 'https://api.binance.com/api/v3/ticker/price' # get current price
order_endpoint = 'https://api.binance.com/api/v3/order'  # orders endpoint

### Make Trade
Get balances and prices and then trade based on this balances and prices.

In [15]:
# get btc balances from account and current prices
balances = requests.get(account_endpoint, headers=api_key, 
                        params=binance_params(secret=sec_key, timestamp=binance_time(), recvWindow=6000)).json()   
btc_balance = float([balance for balance in balances['balances'] if balance['asset'] == 'BTC'][0]['free'])
usdt_balance = float([balance for balance in balances['balances'] if balance['asset'] == 'USDT'][0]['free'])
btc_price = float(requests.get(price_endpoint, params={'symbol': 'BTCUSDT'}).json()['price'])

In [16]:
# make order in account depending on balance and price pred
# buy BTC order with USDT
if pred[0] > 0:
    if round(btc_balance, 4) == 0:
        order = requests.post(order_endpoint, headers = api_key, 
                              params=binance_params(secret=sec_key, symbol='BTCUSDT', side='BUY', type='MARKET',
                                                    timestamp=binance_time(), 
                                                    quantity=truncate(usdt_balance/btc_price, 6)))
    else:
        print('BTC to increase in price but position already in BTC')
        
# Sell BTC into USDT
if pred[0] < 0:
    if round(btc_balance, 4) > 0:
        order = requests.post(order_endpoint, headers = api_key, 
                              params=binance_params(secret=sec_key, symbol='BTCUSDT', side='SELL', type='MARKET',
                                                    timestamp=binance_time(), 
                                                    quantity=truncate(btc_balance, 6)))
        
    else:
        print('BTC to decrease but position already in USD')

BTC to decrease but position already in USD
